In [ ]:
CATALOGO_DESTINO = "spotify_analytics"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_tracks"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

### Configuração

Define catálogo, schema e nome da tabela Silver de tracks (músicas).

### DDL - Tabela de Tracks

Cria tabela Silver com:
- **track_id** (PK): Identificador único da track no Spotify
- **track_name**: Nome da música
- **album_name**: Nome do álbum
- **album_type**: Tipo (album, single, compilation)
- **release_date**: Data de lançamento normalizada
- **release_year**: Ano extraído para agregações
- **popularity**: Score de 0-100
- **duration_ms**: Duração em milissegundos
- **explicit**: Flag conteúdo explícito
- Colunas de metadata (dt_ingestion, dc_origem)

In [ ]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    track_id STRING NOT NULL COMMENT 'Identificador único da track no Spotify (Chave Primária)',
    track_name STRING COMMENT 'Nome da track/música',
    album_name STRING COMMENT 'Nome do álbum ao qual a track pertence',
    album_type STRING COMMENT 'Tipo do álbum: album, single ou compilation',
    release_date DATE COMMENT 'Data de lançamento do álbum (normalizado de vários formatos)',
    release_year INT COMMENT 'Ano de lançamento extraído de release_date',
    popularity INT COMMENT 'Score de popularidade da track (0-100)',
    duration_ms BIGINT COMMENT 'Duração da track em milissegundos',
    explicit BOOLEAN COMMENT 'Indica se a track possui conteúdo explícito',
    dt_ingestion TIMESTAMP COMMENT 'Data/hora de ingestão no lake',
    dc_origem STRING COMMENT 'Origem dos dados (ex: spotify_api_search)',
    CONSTRAINT pk_tracks PRIMARY KEY (track_id)
)
USING DELTA
COMMENT 'Tabela silver: dados limpos e validados de tracks do Spotify'
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true',
    'quality' = 'silver',
    'grain' = 'one_row_per_track'
);
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")